Iris flowers with 4 features (sepal length, sepal width, petal length, petal width) and 3 classes.


# Sample data with different scales
X_train = np.array([
    [5.1, 3.5, 1.4, 0.2],
    [7.0, 3.2, 4.7, 1.4],
    [6.3, 3.3, 6.0, 2.5],
    # ... more samples
])

y_train = np.array([0, 1, 2])  # 3 classes



In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader


x = torch.rand(1000, 4)
y = torch.randint(low=0, high=3, size=(1000,))

# y = F.one_hot(y, num_classes=3).float() not needed because the CrossEntropyLoss doesnt expect the one_hot encoding


x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)



x_train = x_train.to('mps')
x_val = x_val.to('mps')
y_train = y_train.to('mps')
y_val = y_val.to('mps')



train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_val, y_val)



train_data = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = DataLoader(test_data, batch_size=64)


class IrisFlower(nn.Module):
    def __init__(self):
        super(IrisFlower, self).__init__()
        self.f1 = nn.Linear(4, 128)
        self.f2 = nn.Linear(128, 32)
        self.output = nn.Linear(32, 3)


        self.relu = nn.ReLU()
        # self.softmax = nn.Softmax() don't use this as crossentropyloss already has this 

    def forward(self, x):
        x = self.relu(self.f1(x))
        x = self.relu(self.f2(x))
        x = self.output(x)


        return x
    
model = IrisFlower()

model = model.to('mps')

model.train()
epochs = 50 
crit = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.002)

for i in range(epochs):
    t_loss = 0
    for xi, yi in train_data:
        optim.zero_grad()
        pred = model(xi)
        loss = crit(pred, yi)
        loss.backward()
        optim.step()
        t_loss += loss.item()

print(t_loss/len(train_data))

model.eval()
with torch.no_grad():
    test_loss = 0
    correct = 0
    for xi, yi in test_data:
        pred = model(xi)
        correct += (yi == torch.argmax(pred, dim=1)).sum()
        test_loss += crit(pred, yi).item()
        
print("accuracy", correct/len(y_val))
print(test_loss/len(test_data))






tensor([[0.9535, 0.4303, 0.2378, 0.9500],
        [0.0187, 0.7201, 0.1992, 0.7104],
        [0.3909, 0.0475, 0.6138, 0.4889],
        ...,
        [0.7936, 0.0446, 0.6177, 0.9778],
        [0.0261, 0.4802, 0.1942, 0.0934],
        [0.0644, 0.9065, 0.3381, 0.6590]])


KeyboardInterrupt: 